For Udacity Linear Algebra Refresher

In [2]:
from decimal import Decimal, getcontext
getcontext().prec = 30
import operator
import math


In [11]:
class Vector(object):
    CANNOT_NORMALIZE_ZERO_VECTOR_MSG = 'Cannot normalize the zero vector' 
    NO_UNIQUE_ORTHOGONAL_COMPONENT_MSG = 'Cannot project with zero vector'
    NO_UNIQUE_PARALLEL_COMPONENT_MSG = 'Cannot be parallel with zero vector'
    CANNOT_DO_CROSS_PRODUCT_IN_HIGHER_DIMENSIONS_MSG = 'Cannot do cross product in greater than 3 dimensions' 

    def __init__(self, coordinates):
        try:
            self.index = 0
            if not coordinates:
                raise ValueError
            self.coordinates = tuple([Decimal(x) for x in coordinates])
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')


    def __iter__(self):
        return self

    def __next__(self):
        try:
            result = self.coordinates[self.index]
        except IndexError:
            raise StopIteration
        self.index += 1
        return result
  
    def __getitem__(self,index):
        return self.coordinates[index]
    
    def __setitem__(self,index,value):
        self.coordinates[index] = value
            
    def __str__(self):
        return 'Vector: {}'.format(self.coordinates)


    def __eq__(self, v):
        return self.coordinates == v.coordinates
    
    def add(self, v):
        return Vector(tuple(map(operator.add, self.coordinates, v.coordinates)))

    def sub(self, v):
        return Vector(tuple(map(operator.sub, self.coordinates, v.coordinates)))

    def scalar_multiply(self, n):
        # replicate up the number to multiply by
        multiply_vector = (n,) * self.dimension
        return Vector(tuple(map(lambda x, n: x *n, self.coordinates, multiply_vector)))
    
    def magnitude(self):
        squared = Vector(tuple(map(operator.mul, self.coordinates, self.coordinates)))
        return Decimal(math.sqrt(sum(squared.coordinates)))

    def normalized(self):
        try: 
            return self.scalar_multiply(Decimal('1.0') / self.magnitude())
        except ZeroDivisionError:
            raise Exception(self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG)
            
            
    def dot(self, v):
        return sum(map(operator.mul, self.coordinates, v.coordinates))

    def angle_radians(self, v):
        return (self.angle_with(v, in_degrees=False))

    def angle_degrees(self, v):
        return (self.angle_with(v, in_degrees=True))

    def angle_with(self, v, in_degrees=False): 
        try:
            u1 = self.normalized()
            u2 = v.normalized()
            angle_in_radians = math.acos(round(u1.dot(u2),5))
            if in_degrees:
                degrees_per_radian = 180. / math.pi
                return angle_in_radians * degrees_per_radian
            else:
                return angle_in_radians
            
        except Exception as e:
            if str(e) == self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG:
                raise Exception('Cannot compute and angle with zero length vector')
            else:
                raise e

    def is_zero(self, tolerance=1e-10):
        return self.magnitude() < tolerance

    def is_parallel(self, v):
        return (self.is_zero() or
                v.is_zero() or
                round(self.angle_radians(v),5) == round(math.pi,5) or
                self.angle_radians(v) == 0)

    def is_orthogonal(self, v, tolerance = 1e-10):
        return (self.is_zero() or v.is_zero() or (abs(self.dot(v)) < Decimal(tolerance)))
    
    
    def project(self, v):
        u1 = self.normalized()
        scale2 = v.dot(u1)
        return u1.scalar_multiply(scale2)

    def orthovector(self, b):
        projection = self.project(b)
        return b.sub(projection)

    def component_orthogonal_to(self, basis):
        try:
            projection = self.component_parallel_to(basis)
            return self.sub(projection)
        except Exception as e:
            if str(e) == self.NO_UNIQUE_PARALLEL_COMPONENT_MSG:
                raise Exception(self.NO_UNIQUE_ORTHOGONAL_COMPONENT_MSG)
            else:
                raise e

    def component_parallel_to(self, basis):
        try:
            u = basis.normalized()
            weight = self.dot(u)
            return u.scalar_multiply(weight)
        except Exception as e:
            if str(e) == self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG:
                raise Exception(self.NO_UNIQUE_PARALLEL_COMPONENT_MSG)
            else:
                raise e
                
                
    def crossproduct(self, v):
        if self.dimension > 3:
            raise Exception(self.CANNOT_DO_CROSS_PRODUCT_IN_HIGHER_DIMENSIONS_MSG)
        u = Vector([(self.coordinates[1] * v.coordinates[2] - 
                    v.coordinates[1] * self.coordinates[2]),
                   -(self.coordinates[0] * v.coordinates[2] -
                     v.coordinates[0] * self.coordinates[2]), 
                   (self.coordinates[0] * v.coordinates[1] - 
                    v.coordinates[0] * self.coordinates[1])])
        return u
    
    
    def area_of_parallelgram(self, v):
        try:
            xproduct = self.crossproduct(v)
            return xproduct.magnitude()
        except Exception as e:
            if str(e) == self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG:
                raise Exception(self.NO_UNIQUE_PARALLEL_COMPONENT_MSG)
            else:
                raise e
    
    def area_of_triangle(self, v):
        return self.area_of_parallelgram(v) / Decimal(2.0) 

    def cross(self, v):
        try:
            x_1, y_1, z_1 = self.coordinates
            x_2, y_2, z_2 = v.coordinates
            new_coordinates = [   y_1 * z_2 - y_2 * z_1, 
                                -(x_1 * z_2 - x_2 * z_1),
                                  x_1*y_2 - x_2 * y_1 ]  
            return Vector(new_coordinates)
        except ValueError as e:
            msg = str(e)
            if msg == 'need more than 2 values to unpack':
                self_embedded_in_R3 = Vector(self.coordinates + ('0',))
                v_embedded_in_R3 = Vector(v.coordinates + ('0',))
                return self_embedded_in_R3.cross(v_embedded_in_R3)
            elif (msg == 'too many values to unpack' or
                  msg == 'need more than 1 value to unpack'):
                raise Exception(self.CANNOT_DO_CROSS_PRODUCT_IN_HIGHER_DIMENSIONS_MSG)
            else:
                raise e


In [12]:
# Quiz: Coding Cross Products 

v1 = Vector([8.462, 7.893, -8.187])
w1 = Vector([6.984, -5.975, 4.778])
x1 = v1.crossproduct(w1)
print (x1)

v2 = Vector([-8.987, -9.838, 5.031])
w2 = Vector([-4.268, -1.861, -8.866])

print (v2.area_of_parallelgram(w2))

v3 = Vector([1.5, 9.547, 3.691])
w3 = Vector([-6.007, 0.124, 5.772])

print (v3.area_of_parallelgram(w3))
print (v3.area_of_triangle(w3))



Vector: (Decimal('-11.2045709999999977337168388658'), Decimal('-97.6094439999999908463337305875'), Decimal('-105.685161999999993914045148813'))
142.1222214018463319007423706352710723876953125
85.12987479883787500511971302330493927001953125
42.5649373994189375025598565117


In [11]:
v1=Vector([3.039, 1.879])
b1=Vector([0.825, 2.036])
print (v1.project(b1))
print (b1.project(v1))

v2=Vector([-9.88, -3.264, -8.159])
b2 =Vector([-2.155, -9.353, -9.473])
print ("Answer 2")
print (b2.orthovector(v2))

v3 =Vector([3.009, -6.172, 3.692, -2.51])
b3 =Vector([6.404, -9.144, 2.759, 8.718])

v1 = Vector([8.462, 7.893, -8.187])
w1 = Vector([6.984, -5.975, 4.778])
x1 = v1.crossproduct(w1)
print (x1)


v2 = Vector([-8.987, -9.838, 5.031])
w2 = Vector([-4.268, -1.861, -8.866])

print (v2.area_of_parallelgram(w2))

v3 = Vector([1.5, 9.547, 3.691])
w3 = Vector([-6.007, 0.124, 5.772])

print (v3.area_of_parallelgram(w3))
print (v3.area_of_triangle(w3))


Vector: (Decimal('1.50753507130804079404434070770'), Decimal('0.932102138528400303255670923899'))
Vector: (Decimal('1.08260696248446669921320880516'), Decimal('2.67174275832530224589459303452'))
Answer 2
Vector: (Decimal('-8.35008104319576298139037182171'), Decimal('3.37606125428772042918826614535'), Decimal('-1.43374604278118531453872963265'))
Vector: (Decimal('-11.2045709999999977337168388658'), Decimal('-97.6094439999999908463337305875'), Decimal('-105.685161999999993914045148813'))
142.1222214018463319007423706352710723876953125
85.12987479883787500511971302330493927001953125
42.5649373994189375025598565117


In [3]:
# Quiz 5: Coding Vector Projections

print("Question 1")
v1 = Vector([3.039, 1.879])
b1 = Vector([0.825, 2.036])
print(b1.project(v1))

print("Question 2")
v2 = Vector([-9.88, -3.264, -8.159])
b2 = Vector([-2.155, -9.353, -9.473])
print(b2.orthovector(v2))


print("Question 3")
v3 = Vector([3.009, -6.172, 3.692, -2.51])
b3 = Vector([6.404, -9.144, 2.759, 8.718])
print(v3.component_orthogonal_to(b3))
print(v3.component_parallel_to(b3))

Question 1
Vector: (Decimal('1.08260696248446669921320880516'), Decimal('2.67174275832530224589459303452'))
Question 2
Vector: (Decimal('-8.35008104319576298139037182171'), Decimal('3.37606125428772042918826614535'), Decimal('-1.43374604278118531453872963265'))
Question 3
Vector: (Decimal('1.04048383278591013527802116571'), Decimal('-3.36123925156064348251384262723'), Decimal('2.84391503664214983917326352495'), Decimal('-5.18981323325615761241307845876'))
Vector: (Decimal('1.96851616721408976169328214908'), Decimal('-2.81076074843935622616363571113'), Decimal('0.848084963357850331356993057470'), Decimal('2.67981323325615782557589918679'))


In [97]:
# Quiz 4: Checking for Parallel, Orthogonal

print("Question 1")
v1 = Vector([-7.579, -7.88])
w1 = Vector([22.737, 23.64])
print("Vector 1 is parallel to Vector 2")
print(v1.is_parallel(w1))
print("Vector 1 is orthogonal to Vector 2")
print(v1.is_orthogonal(w1))

print("Question 2")
v2 = Vector([-2.029, 9.97, 4.172])
w2 = Vector([-9.231, -6.639, -7.245])
print("Vector 1 is parallel to Vector 2")
print(v2.is_parallel(w2))
print("Vector 1 is orthogonal to Vector 2")
print(v2.is_orthogonal(w2))
print(v2.dot(w2))

print("Question 3")
v3 = Vector([-2.328, -7.284, -1.214])
w3 = Vector([-1.821, 1.072, -2.94])
print("Vector 1 is parallel to Vector 2")
print(v3.is_parallel(w3))
print("Vector 1 is orthogonal to Vector 2")
print(v3.is_orthogonal(w3))

print("Question 4")
v4 = Vector([2.118, 4.827])
w4 = Vector([0,0])
print("Vector 1 is parallel to Vector 2")
print(v4.is_parallel(w4))
print("Vector 1 is orthogonal to Vector 2")
print(v4.is_orthogonal(w4))


Question 1
Vector 1 is parallel to Vector 2
True
Vector 1 is orthogonal to Vector 2
False
Question 2
Vector 1 is parallel to Vector 2
False
Vector 1 is orthogonal to Vector 2
False
-77.6872710000000059639333471751
Question 3
Vector 1 is parallel to Vector 2
False
Vector 1 is orthogonal to Vector 2
True
Question 4
Vector 1 is parallel to Vector 2
True
Vector 1 is orthogonal to Vector 2
True


In [67]:
# Quiz 3: Coding Dot Product & Angle

print("Question 1")
v1 = Vector([7.887, 4.138])
w1 = Vector([-8.802, 6.776])
print(v1.dot(w1))

print("Question 2")
v1 = Vector([-5.955, -4.904, -1.874])
w1 = Vector([-4.496, -8.755, 7.103])
print(v1.dot(w1))

print("Question 3")
v1 = Vector([3.183, -7.627])
w1 = Vector([-2.668, 5.319])
print(v1.angle_radians(w1))

print("Question 4")
v1 = Vector([7.35, 0.221, 5.188])
w1 = Vector([2.751, 8.259, 3.985])
print(v1.angle_degrees(w1))


Question 1
-41.3822859999999945439839166283
Question 2
56.3971780000000056997571107331
Question 3
3.0720263098372476
Question 4
60.27581120523091


In [55]:
# Quiz Lesson 2 

print("Question 1")
v1 = Vector([8.218, -9.341])
v2 = Vector([-1.129, 2.111])
print(v1.add(v2))

print("Question 2")
v1 = Vector([7.119, 8.215])
v2 = Vector([-8.223, 0.878])
print(v1.sub(v2))

print("Question 3")
v1 = Vector([1.671, -1.012, -0.318])
print(v1.scalar_multiply(Decimal(7.41)))

Question 1
Vector: (Decimal('7.08899999999999996802557689080'), Decimal('-7.22999999999999909405801190587'))
Question 2
Vector: (Decimal('15.3420000000000005258016244625'), Decimal('7.33699999999999985522691758888'))
Question 3
Vector: (Decimal('12.3821100000000005402078784300'), Decimal('-7.49892000000000022279067479758'), Decimal('-2.35638000000000008138822948922'))


In [57]:
# Quiz 2 Lesson 2 (Coding Magnitude & Direction)

print("Question 1")
v1 = Vector([-0.221, 7.437])
print(v1.magnitude())

print("Question 2")
v1 = Vector([8.813, -1.331, -6.247])
print(v1.magnitude())

print("Question 3")
v1 = Vector([5.581, -2.136])
print(v1.normalized())

print("Question 4")
v1 = Vector([1.996, 3.108, -4.554])
print(v1.normalized())

Question 1
7.4402829247280646285389593685977160930633544921875
Question 2
10.8841875672922885343041343730874359607696533203125
Question 3
Vector: (Decimal('0.933935214086640295130539147343'), Decimal('-0.357442325262329983594964055642'))
Question 4
Vector: (Decimal('0.340401295943301353537171045562'), Decimal('0.530043701298487295255023200306'), Decimal('-0.776647044952802835008995686630'))


In [19]:
from decimal import Decimal, getcontext

getcontext().prec = 30


class Line(object):

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 2

        if not normal_vector:
            all_zeros = ['0']*self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = normal_vector

        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)

        self.set_basepoint()


    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = ['0']*self.dimension

            initial_index = Line.first_nonzero_index(n)
            initial_coefficient = n[initial_index]

            basepoint_coords[initial_index] = c/initial_coefficient
            self.basepoint = Vector(basepoint_coords)

        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e


    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Line.first_nonzero_index(n)
            terms = [write_coefficient(n[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                     for i in range(self.dimension) if round(n[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output

    def intersection(self, otherline):
        
        return self
    
    def is_parallel(self, otherline):
        return(self.normal_vector.is_parallel(otherline.normal_vector))
    
    def is_overlapping(self, otherline):
        return true
    
    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Line.NO_NONZERO_ELTS_FOUND_MSG)


class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps

In [20]:
l1 = Line(Vector([4.046, 2.836]), 1.21)
l2 = Line(Vector([10.115, 7.09]), 3.025)

print(l1.is_parallel(l2))

Line(Vector([7.204, 3.182]), 8.68)
Line(Vector([8.172, 4.114]), 9.883)

Line(Vector([1.182, 5.562]), 6.744)
Line(Vector([1.773, 8.343]), 9.525)


True
